<a href="https://colab.research.google.com/github/micla321/BAP/blob/master/PB%20Laurent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler


In [2]:
# Specify the file path
file_path ="/content/drive/MyDrive/AFK_H1.xlsx"

data = pd.read_excel(file_path)
data.describe()

,input_1,input_2,output
count,1.000000e+05,1.000000e+05,1.000000e+05
mean,-2.722269e+13,1.790639e+13,9.771850e+31
std,6.916098e+15,7.063002e+15,6.513147e+32
min,-9.758854e+16,-9.811576e+16,5.100057e-38
25%,-2.529802e-01,-1.465311e-01,5.840816e-02
50%,3.076774e-19,3.051135e-19,1.478026e+13
75%,2.274812e-01,1.977993e-01,1.154886e+24
max,9.915818e+16,9.786339e+16,9.832345e+33


In [3]:
# Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Separate the input features and the output variable
input_features = data[['input_1', 'input_2']]
output_variable = data['output']

# Perform standard scaling on the input features
scaler = StandardScaler()
scaled_input_features = scaler.fit_transform(input_features)

# Perform logarithmic transformation on the output variable
log_transformed_output = np.log(output_variable)

# Print the transformed data
print(stats.describe(scaled_input_features))
print(stats.describe(log_transformed_output))

DescribeResult(nobs=100000, minmax=(array([-14.10648032, -13.89411451]), array([14.34130832, 13.85331256])), mean=array([-2.20978791e-17, -1.46371804e-17]), variance=array([1.00001, 1.00001]), skewness=array([-0.16695024, -0.12222463]), kurtosis=array([88.99863078, 85.9586919 ]))
DescribeResult(nobs=100000, minmax=(-85.86898181763533, 78.27098552995528), mean=23.669658426762055, variance=1438.2794388052007, skewness=-0.5639553394944913, kurtosis=-0.5915122850270573)


In [5]:
# Convert the data to PyTorch tensors
input_tensor = torch.tensor(scaled_input_features, dtype=torch.float32).to(device)
output_tensor = torch.tensor(log_transformed_output, dtype=torch.float32).to(device)



In [6]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    input_tensor, output_tensor, test_size=0.2, random_state=42
)

x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

In [7]:

class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 500)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500, 1)  # Additional layer
#        self.fc3 = nn.Linear(32, 1)   # Additional layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
#        x = self.relu(x)  # Apply ReLU activation after the second layer
#        x = self.fc3(x)
        return x

# Create the model
model = RegressionModel(input_size=input_tensor.shape[1])
model.to(device)  # Move the model to GPU if available
print(model)

# Define the loss function
loss_function = nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Define the learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)


RegressionModel(
  (fc1): Linear(in_features=2, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=1, bias=True)
)


In [8]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# Define minibatch size
batch_size = 32

# Train the model
num_epochs = 1000
num_batches = len(x_train) // batch_size

for epoch in range(num_epochs):
    total_loss = 0.0

    # Shuffle the training data for each epoch
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    x_train_shuffled = x_train[indices]
    y_train_shuffled = y_train[indices]

    for batch in range(num_batches):
        # Get the current minibatch
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        batch_inputs = x_train_shuffled[start_idx:end_idx]
        batch_targets = y_train_shuffled[start_idx:end_idx]

        # Reshape the target tensor
        batch_targets = batch_targets.reshape(-1, 1)

        # Move the minibatch to GPU if available
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        # Forward pass
        outputs = model(batch_inputs)
        loss = loss_function(outputs, batch_targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Adjust learning rate
    scheduler.step()

    if (epoch+1) % 10 == 0:
        avg_loss = total_loss / num_batches
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}")
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Learning Rate: {scheduler.get_last_lr()}")


Epoch [10/1000], Loss: 1183.237366442871
Epoch [10/1000], Loss: 824.3477783203125, Learning Rate: [0.1]
Epoch [20/1000], Loss: 1158.114089428711
Epoch [20/1000], Loss: 1058.2532958984375, Learning Rate: [0.1]
Epoch [30/1000], Loss: 1177.3227379516602
Epoch [30/1000], Loss: 1188.477783203125, Learning Rate: [0.1]
Epoch [40/1000], Loss: 1156.9727329833984
Epoch [40/1000], Loss: 787.0048217773438, Learning Rate: [0.1]
Epoch [50/1000], Loss: 1158.2169631835939
Epoch [50/1000], Loss: 955.909423828125, Learning Rate: [0.1]
Epoch [60/1000], Loss: 1140.648581201172
Epoch [60/1000], Loss: 1159.6434326171875, Learning Rate: [0.1]
Epoch [70/1000], Loss: 1161.7459077636718
Epoch [70/1000], Loss: 1383.760986328125, Learning Rate: [0.1]
Epoch [80/1000], Loss: 1151.4079425048828
Epoch [80/1000], Loss: 1789.72412109375, Learning Rate: [0.1]
Epoch [90/1000], Loss: 1152.9864003295897
Epoch [90/1000], Loss: 916.0767822265625, Learning Rate: [0.1]
Epoch [100/1000], Loss: 1132.790809387207
Epoch [100/1000]

KeyboardInterrupt: ignored

In [9]:
# Evaluate the model
with torch.no_grad():
    model.eval()
    test_outputs = model(x_test)
    test_loss_log = loss_function(test_outputs, y_test)
    test_loss_lin = loss_function(torch.exp(test_outputs), torch.exp(y_test))
    print(f"Test Loss log: {test_loss_log.item()}", f"Test Loss lin: {test_loss_lin.item()}")


Test Loss log: 1813.875732421875 Test Loss lin: inf


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20000])) that is different to the input size (torch.Size([20000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
